# MusicExpander Prototype2 Workflow

In [1]:
#import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from joblib import dump, load
import datetime


## Spotify credentials

In [2]:
#Initialize SpotiPy with user credentials
# Remove mine when uploading?

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="9f9d46aed10c480b9b68543a2d15e2c9",
    client_secret="0016f56c1cc641159dc9e19ba85bc682"))

## Functions

In [3]:
#functions:

def answer_refiner():
    
    '''
    This is a more complex version of get_input_song
    it asks the user if their song was played by an artist
    if it is the while loop finishes
    if it is not, the loop continues asking questions
    either until the user says it is the right artist
    or until the user quits.
    '''
    #define affirmative and negative answers:
    affirmative=["Yes","YES","yes","yeah","yep","y","Y"]
    negative=["No","NO","no","Nope","NOPE","nope","n","N"]
    attempts=0
    
    #make sure it is the right song:
    status=False
    while status == False:
    
        input_song=input("Tell me the name of a song you like! ")
        second_input=input("Do you mean the song "+song_name_confirmer(input_song)+ " by "+ artist_retriver(input_song)+"? ")
    
        if second_input in affirmative :
            print("Cool! I know which song you mean! ")
            status=True
            decade=get_decade()

        elif second_input in negative and attempts ==0:
            print("OK. Let's try again. Tell me the name again and the authour or album: ")
            attempts+=1
            
        elif second_input in negative and attempts >0:
            third_input=input("This is not working, do you want to stop? ")
            attempts+=1
            if third_input in affirmative:
                print("ok, let's quit, I don't know the songs you like ")
                break
            elif third_input in negative:
                print("ok, yours must be another song. Let's try again: ")
            else:
                print("Sorry, your answer must be \"Yes\" or \"No\". Try again, please ")  
        else:
            print("Sorry, your answer must be \"Yes\" or \"No\". Try again, please ")
    # i need to return decade too
    return input_song  

def get_decate():
    new_decade=str(input("What decade would you like to explore? Format must be 1990 or 1930"))
        

    isValidDate = True
    try :
        datetime.datetime(int(year))
    except ValueError :
        isValidDate = False

    if(isValidDate) :
        print ("Input date is valid ..")
    else :
        print ("Input date is not valid..")

    

def get_input_song():
    '''
    simple function, replaced by answer_refiner in Prototype1 
    '''
    input_song=input("Tell me the name of a song you like! ")
    return input_song


def search_hotlist(input_song, hot_song_df):
    '''
    search the hot songs df titles column for the input song
    if present, return True. Otherwise, return False
    '''
    if len(hot_df.loc[hot_song_df['song titles'] == input_song])!=0:
        return True
        
    else:
        return False
    
    
def hot_song_retriver(input_song, hot_song_df):
    '''
    get a random song form the sd of hot songs
    '''
    
    #get random number within hot_song_df range:
    numb_r=randrange(len(hot_song_df))
    #song_titles are in column 0:
    #get value form "song title" column and numb_r index:
    return(hot_df.iloc[numb_r,0])

def song_name_confirmer(input_song):
    '''
    retrieve the song name of a given song with sp.search
    spotify credentials must be provided above
    to get the full name of the song
    '''
    #search in spotify the info about the input song
    song_info = sp.search(input_song,limit=1)
    #save the song name
    song_name=(song_info["tracks"]["items"][0]["name"])
    return song_name

def artist_retriver(input_song):
    '''
    retrieve the artist of a given song with sp.search
    spotify credentials must be provided above
    '''
    #search in spotify the info about the input song
    song_info = sp.search(input_song,limit=1)
    #save the artist's name
    artists_names=(song_info["tracks"]["items"][0]["artists"][0]["name"])
    return artists_names

def get_song_features(song_uri):
    '''
    input a individual song uri
    output a df with a single row,
    and the features in columns
    '''
    features_dict=sp.audio_features(song_uri)
    features_df=pd.DataFrame(features_dict)
    
    return features_df

def get_a_clusted_song(greatest_funct_df,song_cluster): #add decade
    '''
    from the rows that have a "clusters" value equal to the user's song value
    get 1 row with pd.sample
    return a one line df
    '''
    #search the big df with info of the songs and the cluster
    # from the rowns with the appropiate cluster sample 1 and return
    song_cluster=int(song_cluster)
    
    df_answer = greatest_funct_df[(greatest_funct_df['clusters'] == song_cluster)].sample()
    #add decade filter # greatest_funct_df[(greatest_funct_df["release year"]== decade)]
    return df_answer

def get_info_on_user_song(user_input):
    '''
    search spotify for any info the user has on the song
    '''
    song_info = sp.search(user_input,limit=1)
    song_uri=song_info["tracks"]["items"][0]["uri"]
    song_id=song_info["tracks"]["items"][0]["id"]
    song_artist=song_info["tracks"]["items"][0]["artists"][0]["name"]
    song_name=song_info["tracks"]["items"][0]["name"]
    song_year=song_info["tracks"]["items"][0]["album"]["release_date"][0:4]
    
    songs_dict={
    "song titles":song_name,
    "artists":song_artist,
    "uri":song_uri,
    "id":song_id,
    }
    
    return songs_dict
    
def get_song_uri(user_input):
    '''
    search spotify for the song's uri
    '''
    song_info = sp.search(user_input,limit=1)
    song_uri=song_info["tracks"]["items"][0]["uri"]
    
    return song_uri

def get_song_year(user_input):
    '''
    search spotify for the song's album release
    '''
    song_info = sp.search(user_input,limit=1)
    song_release=song_info["tracks"]["items"][0]["album"]["release_date"]
    
    return song_release


def get_decade():
    '''
    get a year from the user that later will be used as decade
    '''
    year=int(input("What decade would you like to explore? Format must be yyyy for example: 1960: "))
    while year <1900 or year >2022:
        print ("The year was in a wrong format")
        year=int(input("Tell me a decade again. Format must be yyyy like 1990 or 1930: "))
    return year

## Loading Files

In [4]:

#load the hot songs df,
#and the pickles for: original cluster
#and for clustered with only columns used in clustering+ cluster column


#only with clustering columns plus cluster column:
greatest_clustering_df = pd.read_pickle("Data/clustering_df_decade_18.pkl")


#original df with all the columns plus clustering
greatest_df = pd.read_pickle("Data/greatest_clustered_df_decade_18.pkl")

hot_df=pd.read_csv("Data/hot_songs.csv")

#load kmeans and scaler:

Five_kmeans=load("Data/kmeans_decade_18.joblib")

scaler=load("Data/scaler.joblib") 


## The main 

In [6]:
#get a song from the user (make sure it is the right song)
user_song=answer_refiner()

#first park, look if it is in the billboard 100 hot song
#if it is, recomend another from the list with song retriever
if search_hotlist(user_song, hot_df)== True:
    print("You have a trendy taste! You may also like this other song:")
    new_song=hot_song_retriver(user_song, hot_df)
    print(new_song+" by "+ artist_retriver(new_song))
    
else:
    print("Ok! your song is not the trendiest, but let me look for a recomendation for you.\nI can even give you one from your decade choice")
    
    #get id of the song      
    song_id=get_song_uri(user_song)
    #
    your_features_df=get_song_features(song_id)
       
    #drop the columns I dropped for the training:
    your_features_for_clustering_df=your_features_df.drop(columns=[ 
                "type",
                "id",
                "uri",
                "track_href",                
                "analysis_url",
                "duration_ms",
                "time_signature"])
    
    # scale the song features (transform) with the imported sclaer
    song_scaled = scaler.transform(your_features_for_clustering_df)
    # assign the song to a cluster (predict) with the imported kmeans model
    clusters=Five_kmeans.predict(song_scaled)
    #store the cluster number in your_cluster
    your_cluster=clusters
    #use your decade
    
    answer_df=get_a_clusted_song(greatest_df,your_cluster) #add decade
    answer_title=(answer_df.loc[:,"song titles"].iloc[0])
    answer_artist=(answer_df.loc[:,"artists"].iloc[0])
    answer_link=(answer_df.loc[:,"uri"].iloc[0])
    answer_id=(answer_df.loc[:,"song id"].iloc[0])
    #get year of album release:
    release_year=sp.track(answer_id)["album"]["release_date"][0:4]
    
    print("My recomendation is",answer_title," from ", answer_artist, "released on",release_year)

    
    

Tell me the name of a song you like! con altura
Do you mean the song Con Altura by ROSALÍA? yes
Cool! I know which song you mean! 
What decade would you like to explore? Format must be yyyy for example: 1960: 1930
Ok! your song is not the trendiest, but let me look for a recomendation for you.
I can even give you one from your decade choice
My recomendation is Atmosphere Strut  from  Cloud One released on 1976


In [6]:
#My tests:
    #song_id="spotify:track:015Pp0xssDnMGLQ8pVMIaw" #corazon de mimbre  cl 4, 1
    #song_id="spotify:track:2vfINZ5Xnf1HggzoQKgqps" #dream a little dream cl 1,2
    #song_id="spotify:track:4WGHQuR8gGRjAd5OjrqRHf" #morning clasica cl 0, 3 
    #song_id="spotify:track:7c9sDyogsPK8XUAubCK69Y" #vicio  3, 4
    #song_id="spotify:track:0wDNAKDSmUSWJDMYHnqmCL" #la vie en rose 1, 2
    #song_id="spotify:track:18o9xce1pADBc0fmGiPXmx" # danubio azul 0, 3
    #song_id="spotify:track:4pbJqGIASGPr0ZpGpnWkDn" #we will rock you 4, 1
    #song_id="spotify:track:1pZn8AX1WulW8IO338hE5D"# blue moon  1, 2
    #song_id="spotify:track:2Fxmhks0bxGSBdJ92vM42m" #bad guy billie eilish 2, 0
    #song_id="spotify:track:2fICdpdRotwfmGYzvs8Ngf" #rock you like a hurricane 3, 4
    #song_id="spotify:track:2qG5sZ7Si6sdK74qLxedYM"#con altura 4, 1
    #song_id="spotify:track:6jEZLz3YpnEBRpVkv35AmP"# gasolina 4, 1
    #song_id="spotify:track:0tjZv2hChdHZCW1zFXpy1J" # bad bunny, 0
    #Your Return (He Ri Jun Zai Lai) by 周璇  with L'italiana in Algeri
    